<a href=https://docs.google.com/uc?id=1xbyIyDFmryHaZ4HhVNDBeoMCZGyXCf10 target="_blank"><img src=https://docs.google.com/uc?id=1xbyIyDFmryHaZ4HhVNDBeoMCZGyXCf10 
width="150" border="10" /></a>

# Basic operations using Nexus v0

##  General introduction

At the core of Blue Brain Nexus lies a knowledge graph [(What is a knowledge graph?)](https://hackernoon.com/wtf-is-a-knowledge-graph-a16603a1a25f). The Nexus KnowledgeGraph operates on **4 types** of resources: **Organizations, Domains, Schemas** and **Instances**, nested as described in the diagram below:

<a href= target=https://bbp-nexus.epfl.ch/staging/docs/kg/assets/api-reference/resources.png "_blank"><img src=https://bbp-nexus.epfl.ch/staging/docs/kg/assets/api-reference/resources.png 
width="500" border="10" /></a>


The Nexus KnowledgeGraph exposes a RESTful interface over HTTP(S). The generally adopted transport format is **JSON-LD**. All resources in the system generally follow the very same lifecycle (see diagram below). Changes to the data (creation, updates, state changes) are recorded into the system as **revisions**.

<a href= target=https://bbp-nexus.epfl.ch/staging/docs/kg/assets/api-reference/resource-lifecycle.png "_blank"><img src=https://bbp-nexus.epfl.ch/staging/docs/kg/assets/api-reference/resource-lifecycle.png 
width="500" border="10" /></a>

[Nexus API documentation](https://bbp-nexus.epfl.ch/dev/docs/kg/index.html)

## Objective of this notebook

This Jupyter notebook will guide you through some basic operations to create and manage resources with Nexus. This includes:

* Creating a domain on Nexus
* Creating a schema
* Publishing a schema
* Creating a data instance
* Updating the data instance
* Attaching a file to that data instance 
* Filtering data instances by their type

These operations will be performed in the **sandbox** organization of the staging environment of Blue Brain Nexus.

## The Nexus client Pyxus

Nexus exposes a RESTful interface over HTTP(S). To faciliate API interactions, the Nexus client [Pyxus](https://github.com/HumanBrainProject/pyxus) can be used to access and manage resources in Nexus.

**Execute the following code to install Pyxus and other dependencies:**

In [11]:
!pip install --upgrade --extra-index-url https://testpypi.python.org/pypi "pyxus==0.5.1" pyyaml requests

Looking in indexes: https://pypi.org/simple, https://testpypi.python.org/pypi
Requirement already up-to-date: pyxus==0.5.1 in /Applications/Anaconda/anaconda/envs/pyshacl/lib/python3.6/site-packages (0.5.1)
Requirement already up-to-date: pyyaml in /Applications/Anaconda/anaconda/envs/pyshacl/lib/python3.6/site-packages (3.13)
Requirement already up-to-date: requests in /Applications/Anaconda/anaconda/envs/pyshacl/lib/python3.6/site-packages (2.19.1)
Requirement not upgraded as not directly required: rdflib-jsonld in /Applications/Anaconda/anaconda/envs/pyshacl/lib/python3.6/site-packages (from pyxus==0.5.1) (0.4.0)
Requirement not upgraded as not directly required: pystache in /Applications/Anaconda/anaconda/envs/pyshacl/lib/python3.6/site-packages (from pyxus==0.5.1) (0.5.4)
Requirement not upgraded as not directly required: pyld in /Applications/Anaconda/anaconda/envs/pyshacl/lib/python3.6/site-packages (from pyxus==0.5.1) (0.8.2)
Requirement not upgraded as not directly required: o

**Import relevant Pyxus classes and required Python modules:**

In [3]:
import requests
import yaml

from pyxus.client import NexusClient
from pyxus.resources.entity import Domain
from pyxus.resources.entity import Schema
from pyxus.resources.entity import Instance
from openid_http_client.auth_client.access_token_client import AccessTokenClient

#### Initialize an access token client for anonymised access:

In [22]:
auth_client = None

#### Initialize an access token client for an authenticated access :

In [25]:
token ="put your token here"
auth_client = AccessTokenClient(token)

#### Set up a Nexus client by executing the code below:

In [26]:
client = NexusClient(scheme="https", 
                     host="bbp-nexus.epfl.ch", 
                     prefix="staging/v0", 
                     alternative_namespace="https://bbp-nexus.epfl.ch",
                     auth_client=auth_client)

In [27]:
your_domain = client.domains.read("ngv", "core")
print("Your domain identifer is {}".format(your_domain.data["@id"]))

Your domain identifer is https://bbp-nexus.epfl.ch/staging/v0/domains/ngv/core


## Basic operations

### Create a custom domain on Nexus using Pyxus

Inside an organization on Nexus, you can create your own custom domains. To set up your own domain in the sandbox organization, input a custom name and description for your domain below and execute the cell:

In [ ]:
domain_name = "Your domain name"
domain_description = "Your domain description"

With the provided domain name and description, you can **create your domain** on Nexus:

In [ ]:
organization_name = "sandbox"
your_domain = Domain.create_new(organization_name, domain_name, domain_description) 
client.domains.create(your_domain)

You can access your domain identifier through the link provided after executing the code below:

In [ ]:
your_domain = client.domains.read(organization_name, domain_name)
print("Your domain identifer is {}".format(your_domain.data["@id"]))

You can also check out your newly created domain in the [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### Create and publish a schema

Now that you have created your custom domain in Nexus, you can create a **schema** (written in the so-called [Shapes Constraint Language](https://www.w3.org/TR/shacl/) or SHACL) inside that domain - e.g. a schema to describe a person with the properties given name, family name and e-mail address (note: you cannot 'delete' a schema once created).

In [ ]:
example_person_schema = {
    "@context": [
        "https://bbp-nexus.epfl.ch/staging/v0/contexts/nexus/core/schema/v0.2.0"
    ],
    "@type": "nxv:Schema",
    "shapes": {
        "@type": "sh:NodeShape",
        "description": "schema.org person description.",
        "nodeKind": "sh:BlankNodeOrIRI",
        "targetClass": "schema:Person",
        "property": [
        {
            "path": "schema:email",
            "datatype": "xsd:string",
            "name": "Email",
            "pattern": "^[^@\\s]+@[^@\\s]+\\.[^@\\s]+$"
        },
        {
            "datatype": "xsd:string",
            "description": "Given name. In the U.S., the first name of a Person. This can be used along with familyName instead of the name property.",
            "name": "givenName",
            "path": "schema:givenName",
            "minCount": "1"
        },
        {
            "datatype": "xsd:string",
            "description": "Family name. In the U.S., the last name of an Person. This can be used along with givenName instead of the name property.",
            "name": "familyName",
            "path": "schema:familyName",
            "minCount": "1"
        }
    ]
  }
}

In [ ]:
schema_name = "person"  # Provide the name for the example schema here
schema_version = "v0.1.0"  # Provide the version for the example schema here
content = example_person_schema

Run the following code to create the example_person_schema in Nexus:

In [ ]:
schema = Schema.create_new(organization=organization_name, 
                           domain=domain_name, 
                           schema=schema_name,
                           version=schema_version, 
                           content=content)
client.schemas.create(schema)

Access the schema identifier of the schema you have just created:

In [ ]:
schema = client.schemas.read(organization=organization_name, 
                             domain=domain_name, 
                             schema=schema_name, 
                             version=schema_version)
print("The schema identifier is {}".format(schema.data["@id"]))

To be able to submit data against your schema, it has to be published. To publish your schema, run the following code:

In [ ]:
client.schemas.publish(entity=schema,publish=True)

Check out your newly created and published schema in [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### Create a data instance against your schema

Now that your schema is stored and published, we can validate data against the schema and store it in Nexus. 

In [ ]:
example_person_data = {
    "@context": {
        "Person": "http://schema.org/Person",
        "givenName": "http://schema.org/givenName",
        "familyName": "http://schema.org/familyName"
    },
    "@type": [
        "Person"
    ],
    "familyName": "Nexus",
    "givenName": "Brian"
}

In [ ]:
instance = Instance.create_new(organization=organization_name, 
                           domain=domain_name, 
                           schema=schema_name,
                           version=schema_version, 
                           content=example_person_data)
client.instances.create(instance)

Check out your newly created data instance in [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### Update your data instance

After data has been stored in Nexus, it can be updated (e.g. if you want to provide a more detailed description such as an e-mail address or correct a typo).

In [ ]:
updated_example_person_data = {
    "@context": {
        "Person": "http://schema.org/Person",
        "givenName": "http://schema.org/givenName",
        "familyName": "http://schema.org/familyName",
        "email": "http://schema.org/email"
    },
    "@type": [
        "Person"
    ],
    "familyName": "Nexus",
    "givenName": "Brian",
    "email": "brian@nexus.ch"
}

Using the updated payload, you can update your instance on Nexus:

In [ ]:
instance.data = updated_example_person_data
client.instances.update(entity=instance)

Check out your updated data instance in [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/) (note how the version number has changed).

### Attach a file to your data instance

Nexus allows one to store metadata as well as file attachments. To attach a binary file to your data instance, execute the following code:

In [ ]:
example_person_data_id = instance.data["@id"]
example_person_data_rev = instance.get_revision()

In [ ]:
filepath = "https://docs.google.com/uc?id=1V8-hGYNMVqlCIrvlTKTxseMfZSGEqbHl"  # Provide the address of the file you want to attach here
r = requests.get(filepath)
file =  r.content

url = "{}/attachment?rev={}".format(example_person_data_id, example_person_data_rev)
file_attachment = {'file': file}
response = requests.put(url, files=file_attachment)

Check out your data instance with the attachment using [Nexus Explorer](https://bbp-nexus.epfl.ch/staging/explorer/)

### Query Nexus to get instances of type schema:Person

Now that you have created your schema and validated a data instance against it, you can query Nexus to retrieve data. 

In [ ]:
query_filter = {
  "@context": "https://bbp-nexus.epfl.ch/staging/v0/contexts/neurosciencegraph/core/data/v0.1.2",
  "filter": {
    "op": "eq",
    "path": "rdf:type",
    "value": "schema:Person"
  }
}

Execute the code below to see how many instances of type schema:Person are in Nexus:

In [ ]:
response = requests.post("https://bbp-nexus.epfl.ch/staging/v0/queries", 
                         json=query_filter, 
                         allow_redirects=False)
get_response = requests.get(response.headers["Location"])
print("There are a total of {} instances of type schema:Person in Nexus".format(yaml.load(get_response.text)["total"]))

**THE END**